<a href="https://colab.research.google.com/github/malinphy/recommender_sys/blob/main/convolutional_recommender_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet d2l==0.17.0 ;
!pip install --quiet -U mxnet-cu101==1.7.0 ;
!pip install --quiet tensorflow-datasets ;
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import random 
import random
import mxnet as mx
from mxnet import gluon, np, npx
from mxnet.gluon import nn
from d2l import mxnet as d2l
# import tensorflow_datasets as tfds

import tensorflow as tf 
from tensorflow import keras 
# from tensorflow.keras import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import MaxPool1D
from tensorflow.keras.layers import Dropout,Lambda

     |████████████████████████████████| 83 kB 1.0 MB/s 
     |███████████████████████████████▌| 834.1 MB 1.2 MB/s eta 0:00:10tcmalloc: large alloc 1147494400 bytes == 0x560943850000 @  0x7f257294e615 0x56090a5144cc 0x56090a5f447a 0x56090a5172ed 0x56090a608e1d 0x56090a58ae99 0x56090a5859ee 0x56090a518bda 0x56090a58ad00 0x56090a5859ee 0x56090a518bda 0x56090a587737 0x56090a609c66 0x56090a586daf 0x56090a609c66 0x56090a586daf 0x56090a609c66 0x56090a586daf 0x56090a519039 0x56090a55c409 0x56090a517c52 0x56090a58ac25 0x56090a5859ee 0x56090a518bda 0x56090a587737 0x56090a5859ee 0x56090a518bda 0x56090a586915 0x56090a518afa 0x56090a586c0d 0x56090a5859ee
     |████████████████████████████████| 846.0 MB 20 kB/s 


In [ ]:
TARGET_NUM = 1
batch_size = 4096
L=5
d=16
d_prime=4 
drop_ratio=0.05
num_factors= 10
num_users =943
num_items =1682

In [ ]:
class SeqDataset(gluon.data.Dataset):
    def __init__(self, user_ids, item_ids, L, num_users, num_items,
                 candidates):
        user_ids, item_ids = np.array(user_ids), np.array(item_ids)
        sort_idx = np.array(
            sorted(range(len(user_ids)), key=lambda k: user_ids[k]))
        u_ids, i_ids = user_ids[sort_idx], item_ids[sort_idx]
        temp, u_ids, self.cand = {}, u_ids.asnumpy(), candidates
        self.all_items = set([i for i in range(num_items)])
        [temp.setdefault(u_ids[i], []).append(i) for i, _ in enumerate(u_ids)]
        temp = sorted(temp.items(), key=lambda x: x[0])
        u_ids = np.array([i[0] for i in temp])
        idx = np.array([i[1][0] for i in temp])
        self.ns = ns = int(
            sum([
                c - L if c >= L + 1 else 1
                for c in np.array([len(i[1]) for i in temp])]))
        self.seq_items = np.zeros((ns, L))
        self.seq_users = np.zeros(ns, dtype='int32')
        self.seq_tgt = np.zeros((ns, 1))
        self.test_seq = np.zeros((num_users, L))
        test_users, _uid = np.empty(num_users), None
        for i, (uid, i_seq) in enumerate(self._seq(u_ids, i_ids, idx, L + 1)):
            if uid != _uid:
                self.test_seq[uid][:] = i_seq[-L:]
                test_users[uid], _uid = uid, uid
            self.seq_tgt[i][:] = i_seq[-1:]
            self.seq_items[i][:], self.seq_users[i] = i_seq[:L], uid

    def _win(self, tensor, window_size, step_size=1):
        if len(tensor) - window_size >= 0:
            for i in range(len(tensor), 0, -step_size):
                if i - window_size >= 0:
                    yield tensor[i - window_size:i]
                else:
                    break
        else:
            yield tensor

    def _seq(self, u_ids, i_ids, idx, max_len):
        for i in range(len(idx)):
            stop_idx = None if i >= len(idx) - 1 else int(idx[i + 1])
            for s in self._win(i_ids[int(idx[i]):stop_idx], max_len):
                yield (int(u_ids[i]), s)

    def __len__(self):
        return self.ns

    def __getitem__(self, idx):
        neg = list(self.all_items - set(self.cand[int(self.seq_users[idx])]))
        i = random.randint(0, len(neg) - 1)
        return (self.seq_users[idx], self.seq_items[idx], self.seq_tgt[idx],
                neg[i])

In [ ]:
TARGET_NUM, L, batch_size = 1, 5, 4096
df, num_users, num_items = d2l.read_data_ml100k()
train_data, test_data = d2l.split_data_ml100k(df, num_users, num_items,
                                              'seq-aware')
users_train, items_train, ratings_train, candidates = d2l.load_data_ml100k(
    train_data, num_users, num_items, feedback="implicit")
users_test, items_test, ratings_test, test_iter = d2l.load_data_ml100k(
    test_data, num_users, num_items, feedback="implicit")
train_seq_data = SeqDataset(users_train, items_train, L, num_users, num_items,
                            candidates)
train_iter = gluon.data.DataLoader(train_seq_data, batch_size, True,
                                   last_batch="rollover",
                                   num_workers=d2l.get_dataloader_workers())
test_seq_iter = train_seq_data.test_seq
train_seq_data[0]


(array([0], dtype=int32),
 array([241., 170., 110., 255.,   4.]),
 array([101.]),
 971)

In [ ]:
pos_1 = []
pos_2 = []
pos_3 = []
pos_4 = []
for i in train_seq_data:
  pos_1.append(int(i[0]) )
  pos_2.append((i[1]) )
  pos_3.append(int(i[2]))
  pos_4.append(int(i[3]))
# I believe, in this part seq variable or input is obvious,
# output is single number in the array

def expander(x):
    return (tf.reshape(x,shape= [x.shape[0],x.shape[1],x.shape[2],1], name=None))
    
def seqer(x):
    return (tf.squeeze(x, axis=2))


In [ ]:
big_arr = []
for i in range(len(pos_2)):
  x = pos_2[i]
  small_arr = []
  for j in x:
    # print(int(j))
    small_arr.append(int(j))
  big_arr.append(small_arr)

In [ ]:
len(pos_4)

94342

In [ ]:
# class Caser(nn.Block):
class Caser(tf.keras.Model):
    def __init__(self, num_factors, num_users, num_items, L=5, d=16,
                 d_prime=4, drop_ratio=0.05, **kwargs):
        super(Caser, self).__init__(**kwargs)
        self.P = Embedding(num_users, num_factors)
        self.Q = Embedding(num_items, num_factors)
        self.d_prime = d_prime
        self.d =  d
        # Vertical convolution layer
        # self.conv_v = nn.Conv2D(d_prime, (L, 1), in_channels=1)
        self.conv_v = tf.keras.layers.Conv2D(d_prime,(L,1))
        # Horizontal convolution layer
        h = [i + 1 for i in range(L)]
        # self.conv_h, self.max_pool = nn.Sequential(), nn.Sequential()
        # for i in h:
        #     self.conv_h.add(nn.Conv2D(d, (i, num_factors), in_channels=1))
        #     self.max_pool.add(nn.MaxPool1D(L - i + 1))
        # Fully-connected layer
        self.fc1_dim_v = d_prime * num_factors
        self.fc1_dim_h = d * len(h)
        # self.fc = Dense(d_prime * num_factors + d * L,
        #                    activation='relu')
        self.fc = Dense(num_factors, activation = 'relu')
        self.Q_prime = Embedding(num_items, num_factors * 2)
        self.b = Embedding(num_items, 1)
        self.dropout = Dropout(drop_ratio)
        def third_expander(x):
            y = x[:,:,:,tf.newaxis]
            return(y)

        def third_seqer(x):
            y = tf.squeeze(x, axis = 2)
            return(y)

        self.expander = tf.keras.layers.Lambda(third_expander)
        self.seqer = tf.keras.layers.Lambda(third_seqer)
    def call(self, user_id, seq, item_id):
        h = [i + 1 for i in range(L)]
        # item_embs = np.expand_dims(self.Q(seq), 1)
        item_embs = self.Q(seq)
        user_emb = self.P(user_id)
        out, out_h, out_v, out_hs = None, None, None, []
        if self.d_prime:
            out_v = self.expander(item_embs)
            out_v = self.conv_v(out_v)
            print('ilk out_v',out_v.shape)
            # out_v = out_v.reshape(out_v.shape[0], self.fc1_dim_v)
            out_v = tf.reshape(out_v, shape = (out_v.shape[1], self.fc1_dim_v))
        if self.d:
            for i in h:
              seq_model_L = tf.keras.Sequential([
                self.expander,
                tf.keras.layers.Conv2D(d,(i,num_factors)),
                self.seqer,
                tf.keras.layers.MaxPool1D(L - i +1)
                  ])
            # for i in h :

              
              out_hs.append(seq_model_L(item_embs))

            # for conv, maxp in zip(self.conv_h, self.max_pool):
            #     conv_out = np.squeeze(npx.relu(conv(item_embs)), axis=3)
            #     t = maxp(conv_out)
            #     pool_out = np.squeeze(t, axis=2)
            #     out_hs.append(pool_out)
            out_h = tf.concat(out_hs, axis=1)
        print('out_v', out_v.shape)
        print('out_h', out_h.shape)
        out_h =tf.reshape(out_h, shape=([1, out_h.shape[1]*out_h.shape[2]]))
        print('son_out_h', out_h.shape)
        out = tf.concat([out_v, out_h], axis=1)
        print('deneme_2')
        z = self.fc(self.dropout(out))
        print('z shape', z.shape)
        print('user_emb shape', user_emb.shape)
        user_emb = tf.reshape(user_emb, [1,user_emb.shape[1]*user_emb.shape[2]])
        # user_emb = tf.squeeze(user_emb, [user_emb.shape[-2], user_emb.shape[-1]])
        x = tf.concat([z, user_emb], axis=1)
        print('concat')
        item_embs = self.Q_prime(item_id)
        print('item_Embs_shpa', item_embs.shape)
        q_prime_i = tf.reshape(item_embs, [1,item_embs.shape[1]*item_embs.shape[2]])
        # q_prime_i = np.squeeze(item_embs)
        print('sondan bir')
        id_embs = self.b(item_id)
        b = tf.reshape(id_embs, [1,id_embs.shape[1]*id_embs.shape[2]])
        # b = np.squeeze(self.b(item_id))
        print('son')
        print('X shape', x.shape)
        print('q_prime_i shape', q_prime_i.shape)
        print('b shape', b.shape)
        # res = (x * q_prime_i).sum(1) + b
        res = tf.reduce_sum(x*q_prime_i, 1) +b
        return res


In [ ]:
user_id = tf.keras.Input(shape= (1), dtype = 'float64')
seq = tf.keras.Input(shape= (5), dtype = 'float64')
item_id = tf.keras.Input(shape= (1), dtype = 'float64')

In [ ]:
targets = Caser(num_factors = 10,
      num_users = 943, 
      num_items = 1682, 
      L=5, 
      d=16,
      d_prime=4, 
      drop_ratio=0.05)(user_id,seq, item_id)

caser_model = tf.keras.Model(inputs = [user_id,seq,item_id], outputs = targets)

ilk out_v (None, 1, 10, 4)
out_v (1, 40)
out_h (None, 5, 16)
son_out_h (1, 80)
deneme_2
z shape (1, 10)
user_emb shape (None, 1, 10)
concat
item_Embs_shpa (None, 1, 20)
sondan bir
son
X shape (1, 20)
q_prime_i shape (1, 20)
b shape (1, 1)


In [ ]:
caser_model.compile(
    optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999,
    # epsilon=1e-07,
    # amsgrad=False,
    # name='Adam'), 
    ),
    loss=tf.keras.losses.Hinge(), 
    # loss = 
    metrics=["accuracy"]
)

In [ ]:
caser_model.fit(([tf.constant(pos_1),tf.constant(big_arr),
                  tf.constant(pos_3)]),
                 tf.constant(pos_4),
                epochs =10,
                # batch_size = 5
                )

Epoch 1/10
ilk out_v (None, 1, 10, 4)
out_v (1, 40)
out_h (None, 5, 16)
son_out_h (1, 80)
deneme_2
z shape (1, 10)
user_emb shape (None, 1, 10)
concat
item_Embs_shpa (None, 1, 20)
sondan bir
son
X shape (1, 20)
q_prime_i shape (1, 20)
b shape (1, 1)
ilk out_v (None, 1, 10, 4)


ValueError: ignored